In [ ]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [ ]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

In [ ]:
location_str

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [ ]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

In [ ]:
aq_today_df.info()

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [ ]:
import numpy as np
from datetime import timedelta

In [23]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city

# Wind speed squared
daily_df['wind_speed_10m_max_squared'] = daily_df['wind_speed_10m_max'] ** 2

# Wind speed directions
daily_df['wind_u'] = daily_df['wind_speed_10m_max'] * np.sin(np.radians(daily_df['wind_direction_10m_dominant']))
daily_df['wind_v'] = daily_df['wind_speed_10m_max'] * np.cos(np.radians(daily_df['wind_direction_10m_dominant']))

# Temporal signals
daily_df['day_of_week'] = daily_df['date'].dt.dayofweek          # 0=Mon
daily_df['month'] = daily_df['date'].dt.month
daily_df['is_weekend'] = (daily_df['day_of_week'] >= 5).astype(int)
daily_df['day_of_year'] = daily_df['date'].dt.dayofyear

# Cyclical encoding for day/month so models “feel” seasonality
daily_df['day_of_week_sin'] = np.sin(2 * np.pi * daily_df['day_of_week'] / 7)
daily_df['day_of_week_cos'] = np.cos(2 * np.pi * daily_df['day_of_week'] / 7)
daily_df['month_sin'] = np.sin(2 * np.pi * daily_df['month'] / 12)
daily_df['month_cos'] = np.cos(2 * np.pi * daily_df['month'] / 12)

# Weather-derived interactions
daily_df['precipitation_binary'] = (daily_df['precipitation_sum'] > 0).astype(int)
daily_df['temp_wind_interaction'] = daily_df['temperature_2m_mean'] * daily_df['wind_speed_10m_max']
daily_df['precip_wind_interaction'] = daily_df['precipitation_sum'] * daily_df['wind_speed_10m_max']

daily_df["precip_wind_u"] = daily_df["precipitation_sum"] * daily_df["wind_u"]

# Anomaly weather detection
# get past 30 days of weather data
history = weather_fg.filter(
    (weather_fg.city == city) &
    (weather_fg.date >= today - timedelta(days=30)) &
    (weather_fg.date < today)
).read()

# calculate avg temp of past 30 days
if not history.empty:
    temp_30d_avg = history['temperature_2m_mean'].mean()
else:
    temp_30d_avg = daily_df['temperature_2m_mean'].iloc[0]
daily_df["temperature_30d_avg"] = temp_30d_avg.astype('float64')
# diff between current temp and avg temp of past 30 days
daily_df['temperature_anomaly'] = (daily_df['temperature_2m_mean'] - temp_30d_avg ).astype('float64')
daily_df["temp_anomaly_wind_speed"] = (daily_df["temperature_anomaly"] * daily_df["wind_speed_10m_max"]).astype('float64')

daily_df

Coordinates 59.25°N 18.0°E
Elevation 36.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.01s) 


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,wind_speed_10m_max_squared,wind_u,wind_v,day_of_week,...,day_of_week_cos,month_sin,month_cos,precipitation_binary,temp_wind_interaction,precip_wind_interaction,precip_wind_u,temperature_30d_avg,temperature_anomaly,temp_anomaly_wind_speed
0,2025-11-20,-1.45,0.0,12.605142,1.636545,Stockholm,158.889603,0.359993,12.600000,3,...,-0.900969,-0.5,0.866025,0,-18.277456,0.00000,0.000000,5.761096,-7.211097,-90.896896
1,2025-11-21,-2.35,0.0,16.418526,307.875031,Stockholm,269.567993,-12.959991,10.080012,4,...,-0.900969,-0.5,0.866025,0,-38.583534,0.00000,-0.000000,5.761096,-8.111096,-133.172244
2,2025-11-22,1.50,0.0,17.566378,224.169769,Stockholm,308.577637,-12.240020,-12.599981,5,...,-0.222521,-0.5,0.866025,0,26.349567,0.00000,-0.000000,5.761096,-4.261096,-74.852030
3,2025-11-23,1.55,0.0,13.363711,207.255249,Stockholm,178.588776,-6.119983,-11.880008,6,...,0.623490,-0.5,0.866025,0,20.713753,0.00000,-0.000000,5.761096,-4.211097,-56.275882
4,2025-11-24,-0.30,0.0,1.609969,296.564972,Stockholm,2.592000,-1.440001,0.719998,0,...,1.000000,-0.5,0.866025,0,-0.482991,0.00000,-0.000000,5.761096,-6.061097,-9.758178
5,2025-11-25,0.45,0.1,7.993297,54.162270,Stockholm,63.892792,6.479993,4.680008,1,...,0.623490,-0.5,0.866025,1,3.596983,0.79933,0.647999,5.761096,-5.311097,-42.453171
6,2025-11-26,-1.15,0.0,6.569383,279.462250,Stockholm,43.156788,-6.480001,1.079991,2,...,-0.222521,-0.5,0.866025,0,-7.554790,0.00000,-0.000000,5.761096,-6.911097,-45.401638


In [24]:
# Get historical PM2.5 data (EXCLUDING today to avoid data leakage)
# These features should be computed from past data only, not today's target value
aq_history = air_quality_fg.filter(
    (air_quality_fg.city == city) &
    (air_quality_fg.date >= today - timedelta(days=30)) &
    (air_quality_fg.date < today)  # Exclude today!
).read().sort_values('date')

windows = [1, 2, 3, 7, 14, 21, 30]

if aq_history.empty or len(aq_history) < 2:
    # No history yet, create empty features
    df_feat = pd.DataFrame({
        'city': [city],
        'date': [pd.to_datetime(today)]  # Convert date to timestamp
    })
    for w in windows:
        df_feat[f'pm25_change_{w}d'] = np.nan
        df_feat[f'pm25_std_{w}d'] = np.nan
    # Convert NaN columns to float32 (float) to match schema
    for w in windows:
        df_feat[f'pm25_change_{w}d'] = df_feat[f'pm25_change_{w}d'].astype('float32')
        df_feat[f'pm25_std_{w}d'] = df_feat[f'pm25_std_{w}d'].astype('float32')
else:
    # Compute features from historical data only (NO data leakage)
    # pct_change(periods=w) looks BACKWARD: compares value to w periods ago
    # rolling(window=w) uses the last w values
    # We take .iloc[-1] to get the most recent historical value
    
    # --- PERCENT CHANGE FEATURES (from historical data) ---
    pct_change_features = {
        f"pm25_change_{w}d": aq_history["pm25"].pct_change(periods=w).shift(1).iloc[-1] if len(aq_history) > w else np.nan
        for w in windows
    }
    
    # --- ROLLING STD FEATURES (from historical data) ---
    std_features = {
        f"pm25_std_{w}d": aq_history["pm25"].rolling(window=w, min_periods=1).std().shift(1).iloc[-1] if len(aq_history) >= 1 else np.nan
        for w in windows
    }
    
    # --- COMBINE INTO ONE FEATURES DATAFRAME ---
    df_feat = pd.DataFrame({**pct_change_features, **std_features}, index=[0])
    df_feat['city'] = city
    df_feat["date"] = pd.to_datetime(today)  # Convert date to timestamp


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.60s) 


In [25]:
daily_df = pd.merge(daily_df, df_feat, on=['city', 'date'], how='outer')
# Convert all numeric columns to float32 (float) to match schema
for col in daily_df.columns:
    if col not in ['city', 'date']:
        # Ensure all numeric columns are float32 (float) to match feature group schema
        if pd.api.types.is_numeric_dtype(daily_df[col]):
            daily_df[col] = daily_df[col].astype('float32')

In [ ]:
daily_df.info()

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df)

In [26]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-20 18:11:19,334 INFO: 	36 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286345/fs/1273967/fg/1722116


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286345/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-20 18:11:37,899 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-20 18:11:41,088 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-20 18:13:39,201 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-20 18:13:39,447 INFO: Waiting for log aggregation to finish.
2025-11-20 18:13:51,472 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_change_21d",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762267
         }
       },
       "result": {
         "observed_value": null,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-20T05:11:19.000333Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_colum

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
